### Turn the below block format to code to download the dataset instead of loading.

In [ ]:
#!pip install --upgrade nilearn

from nilearn.datasets import fetch_abide_pcp
# Fetch the full data and update phenotypic data and cross_validation
abide = fetch_abide_pcp(derivatives = ['rois_cc400'], pipeline = 'cpac', quality_checked = True, band_pass_filtering=True, global_signal_regression=True)

y = abide.phenotypic['DX_GROUP']

In [1]:
from numpy import save, load

abide_loaded = load('abide_dataset.npy',allow_pickle=True)
y_loaded = load('labels.npy',allow_pickle=True)

In [2]:
measure = 'correlation'
y_loaded[y_loaded==2] = 0

from nilearn.connectome import ConnectivityMeasure
from nilearn.connectome import sym_matrix_to_vec

conn_est = ConnectivityMeasure(kind = measure)
conn_matrices = conn_est.fit_transform(abide_loaded)
X = sym_matrix_to_vec(conn_matrices)

In [3]:
import numpy as np
from numpy import save
from numpy import load
import tensorflow as tf
from tensorflow import keras
from keras.layers import Concatenate, Input, Conv2D, Dense, concatenate
from keras.models import Model
from sklearn.model_selection import train_test_split
import os

print(tf.__version__)

2.9.0


In [4]:
from random import randrange
from random import seed

def creatingKFolds(dataset,k_folds):
    '''
    This function implements the k fold cross validation algorithm.
    Input: entire dataset with class labels and value of k.
    Output: list containing k divisions of the entire dataset.
    '''
    
    datasetCopy = list(dataset)
    fold_size = int(len(datasetCopy)/k_folds)
    datasetWithFolds = list()
    for _ in range(k_folds):
        fold = []
        while fold_size > len(fold):
            fold.append(datasetCopy.pop(randrange(len(datasetCopy))))
        datasetWithFolds.append(fold)
        
    return datasetWithFolds

class earlyStoppingRepetition(keras.callbacks.Callback):
    def __init__(self):
        self.acc_metrics = list()
    def on_epoch_end(self, epoch, logs):
        self.acc_metrics.append(logs.get('accuracy') + 0.5)
        if epoch > 4:
            evalfunc = self.acc_metrics[-1]+self.acc_metrics[-2]+self.acc_metrics[-3]
            evalfunc = evalfunc/(3*self.acc_metrics[-1])
            if evalfunc == 1:
                print('\nLoss no longer changing. Hence stopping the training!\n')
                self.model.stop_training = True

class EarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if(logs.get('accuracy') >= 0.9):
            print('Ending training since required accuracy achieved.')
            self.model.stop_training = True

In [5]:
dataset = list()
for i in range(len(list(conn_matrices))):
    temp = list()
    temp.append(conn_matrices[i])
    temp.append(y_loaded[i])
    dataset.append(temp)
    
datasetWithKFolds = creatingKFolds(dataset,10)

In [6]:
def layer_setup():
    inp = Input(shape=(392,392,1))

    model1 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (1,392))(inp)
    model1 = tf.keras.layers.Activation('tanh')(model1)
    model1 = tf.keras.layers.MaxPooling2D(1,400)(model1)


    model2 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (2,392))(inp)
    model2 = tf.keras.layers.Activation('tanh')(model2)
    model2 = tf.keras.layers.MaxPooling2D(1,400)(model2)


    model3 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (3,392))(inp)
    model3 = tf.keras.layers.Activation('tanh')(model3)
    model3 = tf.keras.layers.MaxPooling2D(1,400)(model3)


    model4 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (4,392))(inp)
    model4 = tf.keras.layers.Activation('tanh')(model4)
    model4 = tf.keras.layers.MaxPooling2D(1,400)(model4)


    model5 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (5,392))(inp)
    model5 = tf.keras.layers.Activation('tanh')(model5)
    model5 = tf.keras.layers.MaxPooling2D(1,400)(model5)


    model6 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (6,392))(inp)
    model6 = tf.keras.layers.Activation('tanh')(model6)
    model6 = tf.keras.layers.MaxPooling2D(1,400)(model6)


    model7 = tf.keras.layers.Conv2D(filters = 400, kernel_size = (7,392))(inp)
    model7 = tf.keras.layers.Activation('tanh')(model7)
    model7 = tf.keras.layers.MaxPooling2D(1,400)(model7)

    return inp, model1, model2, model3, model4, model5, model6, model7

In [7]:
def model_setup(inp, model1, model2, model3, model4, model5, model6, model7):
    model = tf.keras.layers.concatenate([model1, model2, model3, model4, model5, model6, model7], axis=1)
    model = tf.keras.layers.Dropout(0.25)(model)
    model = tf.keras.layers.Flatten()(model)
    model = tf.keras.layers.Dense(2800, activation='tanh')(model)
    model = tf.keras.layers.Dense(2, activation='softmax')(model)
    model = tf.keras.Model(inputs=inp, outputs=model)

    return model

In [8]:
def model_compile(model):
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
#model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.005), metrics=['accuracy'])

In [9]:
#https://debugah.com/%E3%80%90tensorflow%E3%80%91internalerror-failed-copying-input-tensor-21264/

from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
import gc

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it does something you should see a number as output

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.compat.v1.Session(config=config))

In [10]:
accuracy_metrics = list()

model = True

for ifold in range(len(datasetWithKFolds)):

    reset_keras()
    inp, model1, model2, model3, model4, model5, model6, model7 = layer_setup()
    
    if model:
        del model
        
    model = model_compile(model_setup(inp, model1, model2, model3, model4, model5, model6, model7))
    
    #callbacks = EarlyStopping()

    completeDataset = list(datasetWithKFolds)
    testFold = completeDataset.pop(ifold)

    trainFolds = list()
    for elem in completeDataset:
        trainFolds += elem

    data_test = list()
    labels_test = list()
    for testElem in testFold:
        data_test.append(testElem[0])
        labels_test.append(testElem[1])

    data_train = list()
    labels_train = list()
    for trainElem in trainFolds:
        data_train.append(trainElem[0])
        labels_train.append(trainElem[1])

    data_train = np.array(data_train)
    labels_train = np.array(labels_train)
    data_test = np.array(data_test)
    labels_test = np.array(labels_test)

    model.fit(x=data_train,y=labels_train,batch_size=32,epochs=300)
    accuracy_metrics.append(model.evaluate(data_test, labels_test)[1])

accuracy = sum(accuracy_metrics)/len(accuracy_metrics)
print("Final model accuracy: ",accuracy)

0
Epoch 1/300
25/25 [==============================] - 13s 189ms/step - loss: 10.8079 - accuracy: 0.4930
Epoch 2/300
25/25 [==============================] - 4s 159ms/step - loss: 1.8035 - accuracy: 0.5096
Epoch 3/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9645 - accuracy: 0.4968
Epoch 4/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9229 - accuracy: 0.5121
Epoch 5/300
25/25 [==============================] - 4s 158ms/step - loss: 1.0206 - accuracy: 0.5236
Epoch 6/300
25/25 [==============================] - 4s 166ms/step - loss: 0.9369 - accuracy: 0.5236
Epoch 7/300
25/25 [==============================] - 4s 173ms/step - loss: 1.2138 - accuracy: 0.4943
Epoch 8/300
25/25 [==============================] - 4s 160ms/step - loss: 1.3004 - accuracy: 0.4853
Epoch 9/300
25/25 [==============================] - 4s 170ms/step - loss: 1.5482 - accuracy: 0.4738
Epoch 10/300
25/25 [==============================] - 4s 160ms/step - loss: 1.6807 - ac

25/25 [==============================] - 4s 157ms/step - loss: 4.0990e-05 - accuracy: 1.0000
Epoch 159/300
25/25 [==============================] - 4s 157ms/step - loss: 4.4856e-05 - accuracy: 1.0000
Epoch 160/300
25/25 [==============================] - 4s 157ms/step - loss: 4.3965e-05 - accuracy: 1.0000
Epoch 161/300
25/25 [==============================] - 4s 158ms/step - loss: 4.0265e-05 - accuracy: 1.0000
Epoch 162/300
25/25 [==============================] - 4s 157ms/step - loss: 4.0052e-05 - accuracy: 1.0000
Epoch 163/300
25/25 [==============================] - 4s 158ms/step - loss: 4.2591e-05 - accuracy: 1.0000
Epoch 164/300
25/25 [==============================] - 4s 158ms/step - loss: 3.8265e-05 - accuracy: 1.0000
Epoch 165/300
25/25 [==============================] - 4s 157ms/step - loss: 3.4528e-05 - accuracy: 1.0000
Epoch 166/300
25/25 [==============================] - 4s 157ms/step - loss: 3.8751e-05 - accuracy: 1.0000
Epoch 167/300
25/25 [==============================

25/25 [==============================] - 4s 158ms/step - loss: 1.1506 - accuracy: 0.5223
Epoch 11/300
25/25 [==============================] - 4s 158ms/step - loss: 1.3278 - accuracy: 0.4917
Epoch 12/300
25/25 [==============================] - 4s 158ms/step - loss: 0.9205 - accuracy: 0.5517
Epoch 13/300
25/25 [==============================] - 4s 158ms/step - loss: 1.1778 - accuracy: 0.5351
Epoch 14/300
25/25 [==============================] - 4s 157ms/step - loss: 0.9435 - accuracy: 0.5913
Epoch 15/300
25/25 [==============================] - 4s 158ms/step - loss: 0.5781 - accuracy: 0.6858
Epoch 16/300
25/25 [==============================] - 4s 158ms/step - loss: 0.4888 - accuracy: 0.7650
Epoch 17/300
25/25 [==============================] - 4s 159ms/step - loss: 0.5409 - accuracy: 0.7344
Epoch 18/300
25/25 [==============================] - 4s 159ms/step - loss: 0.4163 - accuracy: 0.7931
Epoch 19/300
25/25 [==============================] - 4s 157ms/step - loss: 0.3043 - accuracy: 

25/25 [==============================] - 4s 157ms/step - loss: 7.8552e-06 - accuracy: 1.0000
Epoch 165/300
25/25 [==============================] - 4s 157ms/step - loss: 7.8347e-06 - accuracy: 1.0000
Epoch 166/300
25/25 [==============================] - 4s 157ms/step - loss: 7.8988e-06 - accuracy: 1.0000
Epoch 167/300
25/25 [==============================] - 4s 157ms/step - loss: 7.2180e-06 - accuracy: 1.0000
Epoch 168/300
25/25 [==============================] - 4s 157ms/step - loss: 6.8515e-06 - accuracy: 1.0000
Epoch 169/300
25/25 [==============================] - 4s 157ms/step - loss: 6.7189e-06 - accuracy: 1.0000
Epoch 170/300
25/25 [==============================] - 4s 157ms/step - loss: 6.8735e-06 - accuracy: 1.0000
Epoch 171/300
25/25 [==============================] - 4s 157ms/step - loss: 6.4200e-06 - accuracy: 1.0000
Epoch 172/300
25/25 [==============================] - 4s 157ms/step - loss: 6.8464e-06 - accuracy: 1.0000
Epoch 173/300
25/25 [==============================

25/25 [==============================] - 4s 157ms/step - loss: 0.8289 - accuracy: 0.4994
Epoch 17/300
25/25 [==============================] - 4s 156ms/step - loss: 1.1116 - accuracy: 0.4866
Epoch 18/300
25/25 [==============================] - 4s 157ms/step - loss: 0.9865 - accuracy: 0.5096
Epoch 19/300
25/25 [==============================] - 4s 157ms/step - loss: 0.8038 - accuracy: 0.4738
Epoch 20/300
25/25 [==============================] - 4s 157ms/step - loss: 1.0069 - accuracy: 0.5351
Epoch 21/300
25/25 [==============================] - 4s 157ms/step - loss: 1.1177 - accuracy: 0.4917
Epoch 22/300
25/25 [==============================] - 4s 157ms/step - loss: 0.8832 - accuracy: 0.5134
Epoch 23/300
25/25 [==============================] - 4s 157ms/step - loss: 1.4821 - accuracy: 0.5262
Epoch 24/300
25/25 [==============================] - 4s 157ms/step - loss: 2.8747 - accuracy: 0.4815
Epoch 25/300
25/25 [==============================] - 4s 157ms/step - loss: 1.6922 - accuracy: 

...

25/25 [==============================] - 4s 157ms/step - loss: 3.0215e-04 - accuracy: 1.0000
Epoch 298/300
25/25 [==============================] - 4s 158ms/step - loss: 4.0759e-04 - accuracy: 1.0000
Epoch 299/300
25/25 [==============================] - 4s 157ms/step - loss: 4.1461e-04 - accuracy: 1.0000
Epoch 300/300
3/3 [==============================] - 0s 69ms/step - loss: 3.5298 - accuracy: 0.5172
4270
Epoch 1/300
25/25 [==============================] - 5s 163ms/step - loss: 8.6579 - accuracy: 0.4981
Epoch 2/300
25/25 [==============================] - 4s 159ms/step - loss: 2.3293 - accuracy: 0.5032
Epoch 3/300
25/25 [==============================] - 4s 163ms/step - loss: 1.2334 - accuracy: 0.5364
Epoch 4/300
25/25 [==============================] - 4s 173ms/step - loss: 0.8529 - accuracy: 0.5415
Epoch 5/300
25/25 [==============================] - 4s 160ms/step - loss: 0.7716 - accuracy: 0.5109
Epoch 6/300
25/25 [==============================] - 4s 162ms/step - loss: 1.1027 -

25/25 [==============================] - 4s 157ms/step - loss: 0.1169 - accuracy: 0.9566
Epoch 156/300
25/25 [==============================] - 4s 157ms/step - loss: 0.1382 - accuracy: 0.9540
Epoch 157/300
25/25 [==============================] - 4s 158ms/step - loss: 0.1150 - accuracy: 0.9540
Epoch 158/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0657 - accuracy: 0.9860
Epoch 159/300
25/25 [==============================] - 4s 158ms/step - loss: 0.0419 - accuracy: 0.9923
Epoch 160/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0283 - accuracy: 0.9949
Epoch 161/300
25/25 [==============================] - 4s 159ms/step - loss: 0.0265 - accuracy: 0.9936
Epoch 162/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0204 - accuracy: 0.9949
Epoch 163/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 164/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0085 - a

Epoch 234/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0029 - accuracy: 0.9987
Epoch 235/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0018 - accuracy: 0.9987
Epoch 236/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0036 - accuracy: 0.9987
Epoch 237/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0027 - accuracy: 0.9987
Epoch 238/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0044 - accuracy: 0.9974
Epoch 239/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0068 - accuracy: 0.9974
Epoch 240/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0047 - accuracy: 0.9974
Epoch 241/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0031 - accuracy: 0.9987
Epoch 242/300
25/25 [==============================] - 4s 157ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 243/300
25/25 [==============================] - 4s 157ms/step - lo

25/25 [==============================] - 4s 163ms/step - loss: 3.8038e-04 - accuracy: 1.0000
Epoch 93/300
25/25 [==============================] - 4s 162ms/step - loss: 3.2573e-04 - accuracy: 1.0000
Epoch 94/300
25/25 [==============================] - 4s 162ms/step - loss: 3.1694e-04 - accuracy: 1.0000
Epoch 95/300
25/25 [==============================] - 4s 163ms/step - loss: 3.1648e-04 - accuracy: 1.0000
Epoch 96/300
25/25 [==============================] - 4s 162ms/step - loss: 2.8328e-04 - accuracy: 1.0000
Epoch 97/300
25/25 [==============================] - 4s 162ms/step - loss: 2.7324e-04 - accuracy: 1.0000
Epoch 98/300
25/25 [==============================] - 4s 163ms/step - loss: 2.7151e-04 - accuracy: 1.0000
Epoch 99/300
25/25 [==============================] - 4s 164ms/step - loss: 2.3198e-04 - accuracy: 1.0000
Epoch 100/300
25/25 [==============================] - 4s 163ms/step - loss: 2.6496e-04 - accuracy: 1.0000
Epoch 101/300
25/25 [==============================] - 4s 

25/25 [==============================] - 4s 161ms/step - loss: 8.5209e-06 - accuracy: 1.0000
Epoch 245/300
25/25 [==============================] - 4s 161ms/step - loss: 8.3240e-06 - accuracy: 1.0000
Epoch 246/300
25/25 [==============================] - 4s 161ms/step - loss: 7.8034e-06 - accuracy: 1.0000
Epoch 247/300
25/25 [==============================] - 4s 161ms/step - loss: 7.7092e-06 - accuracy: 1.0000
Epoch 248/300
25/25 [==============================] - 4s 161ms/step - loss: 7.4778e-06 - accuracy: 1.0000
Epoch 249/300
25/25 [==============================] - 4s 161ms/step - loss: 7.1307e-06 - accuracy: 1.0000
Epoch 250/300
25/25 [==============================] - 4s 162ms/step - loss: 7.9693e-06 - accuracy: 1.0000
Epoch 251/300
25/25 [==============================] - 4s 162ms/step - loss: 7.7362e-06 - accuracy: 1.0000
Epoch 252/300
25/25 [==============================] - 4s 162ms/step - loss: 6.8137e-06 - accuracy: 1.0000
Epoch 253/300
25/25 [==============================

In [11]:
print(accuracy_metrics)

[0.6091954112052917, 0.5632184147834778, 0.6551724076271057, 0.5977011322975159, 0.5632184147834778, 0.517241358757019, 0.5632184147834778, 0.517241358757019, 0.6321839094161987, 0.5517241358757019]


### Turn the below block format to code to see the visualization of high and low connectivity on a brain atlas.